### Imports

In [1]:
import os
import yaml
import polars as pl

### Experiment description

1. Predict ClusterID for clustering datasets
2. Use PointNet
3. No features only x & y data
4. Homogeneous graph

### Variables

In [2]:
input_folder = '../../../data/nieves'
output_folder = '../../../output/nieves/expt1'
input_expt_folder = '../../../data/nieves/expt1'

### Preprocess data to suitable output location

In [44]:
folders = [output_folder, os.path.join(input_folder, "config/expt1"), input_expt_folder]

for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [38]:
# Code specific to dataset & experiment 1
root_dir = os.path.join(input_folder, "ml_data")
for file in os.listdir(root_dir):
    df = pl.read_csv(os.path.join(root_dir, file))
        
    if file.startswith('gt_'):
        df = df.rename({'':'x', '_duplicated_0':'y', 'moleculeClusterIndex': 'gt_label'})
    elif file.startswith('multiple_blinking'):
        df = df.drop(['error', 'moleculeIndex'])
        df = df.rename({'clusterIndex': 'gt_label'})
        
    # change all cluster id's which are != 0 to 1 so we can predict not clustered or clustered
    df = df.with_columns((pl.when(pl.col('gt_label') != 0)).then(1).otherwise(0).alias('gt_label'))
    
    # save 
    file_name, _ = os.path.splitext(file)
    file_name = os.path.join(input_expt_folder, file_name + '.parquet')
    df.write_parquet(file_name)

In [41]:
# Export config variables to file

config = {}

# The following are the names of the 
# x column, y column, z column if present, channel, frame,
# in the csvs being processed
config['x_col'] = 'x'
config['y_col'] = 'y'
config['z_col'] = None
config['channel_col'] = None #'channel'
config['frame_col'] = None # 'frame'

# The number of dimensions to consider
# If 2 only deals with x and y 
# If 3 will read in and deal with z as well (currently not fully supported)
config['dim'] = 2

# choice of which channels user wants to consider
# if null considers all
config['channel_choice'] = None

# specify string for each channel
config['channel_label'] = None
  #0: egfr
  #1: ereg

# whether to not drop the column containing
# pixel
config['drop_pixel_col'] = False

# files to include: all for everyone 
config['include_files'] = 'all' #["mongrad_MoleculeList_12", "mongrad_MoleculeList_11", "mongrad_MoleculeList_10", "mongrad_MoleculeList_1"]

## Choice of input features
config['features'] = []

## GT label per loc or per FOV
config['gt_label_scope'] = 'loc'

## if gt label specified, whih channel is it in (if its label per fov then only one row has to have value), or leave as null
config['gt_label'] = "gt_label"

## specify the gt label map
config['gt_label_map'] = {0: 'unclustered', 1: 'clustered'}

yaml_save_loc = os.path.join(input_folder, 'config/expt1/preprocess.yaml')
with open(yaml_save_loc, "w") as outfile:
        yaml.dump(config, outfile)

In [55]:
# Run the preprocess script using this file

!python ../src/locpix_points/scripts/preprocess.py -i ../../../data/nieves/expt1 -c ../../../data/nieves/config/expt1/preprocess.yaml -o ../../../output/nieves/expt1 -p

List of files which will be processed
['../../../data/nieves/expt1/gt_scenario_8_diffden_MoleculeList_10.parquet', '../../../data/nieves/expt1/gt_scenario_2_normal_50single_MoleculeList_43.parquet', '../../../data/nieves/expt1/gt_scenario_1_CSR_MoleculeList_23.parquet', '../../../data/nieves/expt1/gt_scenario_8_diffden_MoleculeList_22.parquet', '../../../data/nieves/expt1/multiple_blinking_scenario_7_diffsig_DetectionList1.parquet', '../../../data/nieves/expt1/gt_scenario_2_normal_50single_MoleculeList_7.parquet', '../../../data/nieves/expt1/multiple_blinking_scenario_9_dsigsden_new_DetectionList_38.parquet', '../../../data/nieves/expt1/gt_scenario_7_diffsig_MoleculeList_6.parquet', '../../../data/nieves/expt1/gt_scenario_4_lowden_MoleculeList_6.parquet', '../../../data/nieves/expt1/gt_scenario_0_mongrad_MoleculeList_3.parquet', '../../../data/nieves/expt1/multiple_blinking_scenario_4_lowden_DetectionList_34.parquet', '../../../data/nieves/expt1/gt_scenario_1_CSR_MoleculeList_19.parque

/home/oliver/smlm_cloud/locpix-points/src/locpix_points/preprocessing/functions.py:123: UserWarning: All gt labels same for each localisation in file ../../../data/nieves/expt1/gt_scenario_1_CSR_MoleculeList_23.parquet
  warnings.warn(warning)
/home/oliver/smlm_cloud/locpix-points/src/locpix_points/preprocessing/functions.py:123: UserWarning: All gt labels same for each localisation in file ../../../data/nieves/expt1/gt_scenario_1_CSR_MoleculeList_19.parquet
  warnings.warn(warning)
/home/oliver/smlm_cloud/locpix-points/src/locpix_points/preprocessing/functions.py:123: UserWarning: All gt labels same for each localisation in file ../../../data/nieves/expt1/gt_scenario_1_CSR_MoleculeList_40.parquet
  warnings.warn(warning)
/home/oliver/smlm_cloud/locpix-points/src/locpix_points/preprocessing/functions.py:123: UserWarning: All gt labels same for each localisation in file ../../../data/nieves/expt1/gt_scenario_1_CSR_MoleculeList_36.parquet
  warnings.warn(warning)
/home/oliver/smlm_cloud/

/home/oliver/smlm_cloud/locpix-points/src/locpix_points/preprocessing/functions.py:123: UserWarning: All gt labels same for each localisation in file ../../../data/nieves/expt1/gt_scenario_1_CSR_MoleculeList_10.parquet
  warnings.warn(warning)
/home/oliver/smlm_cloud/locpix-points/src/locpix_points/preprocessing/functions.py:123: UserWarning: All gt labels same for each localisation in file ../../../data/nieves/expt1/gt_scenario_1_CSR_MoleculeList_15.parquet
  warnings.warn(warning)
/home/oliver/smlm_cloud/locpix-points/src/locpix_points/preprocessing/functions.py:123: UserWarning: All gt labels same for each localisation in file ../../../data/nieves/expt1/gt_scenario_1_CSR_MoleculeList_22.parquet
  warnings.warn(warning)
/home/oliver/smlm_cloud/locpix-points/src/locpix_points/preprocessing/functions.py:123: UserWarning: All gt labels same for each localisation in file ../../../data/nieves/expt1/gt_scenario_1_CSR_MoleculeList_46.parquet
  warnings.warn(warning)
/home/oliver/smlm_cloud/

### Process data

In [58]:
# Export config variables 

config = {}

# whether to process the data as heterogeneous or homogenous
# graph
config['hetero'] = False

# train/test/val splits
config['train_ratio'] = 0.7
config['val_ratio'] = 0.1
config['test_ratio'] = 0.2

# what to load into position
# Options: xy, xyz
config['pos']= 'xy'

# what to load into features
# Options: 
config["feat"] = None#
    
# label level
# graph or node
config["label_level"] = 'node'

yaml_save_loc = os.path.join(input_folder, 'config/expt1/process.yaml')
with open(yaml_save_loc, "w") as outfile:
        yaml.dump(config, outfile)


In [102]:
# Run the process script 

!python ../src/locpix_points/scripts/process.py -i ../../../output/nieves/expt1 -c ../../../data/nieves/config/expt1/process.yaml

Overwriting metadata...
Train set...
Processing...
Done!
Val set...
Processing...
Done!
Test set...
Processing...
Done!


### Train

In [15]:
# Export config variables 

config = {}

# device to train on (gpu or cpu)
config['gpu'] = True

# model parameters
config['model'] = "pointnetseg"

# optimiser parameters
config['optimiser'] ="adam"
config['lr'] = 0.001
config['weight_decay'] = 0.0001

# training parameters
config['epochs'] = 1
config['batch_size'] = 1
config['num_workers'] = 1 # generall higher -> faster
config['loss_fn'] = "nll"

config['pointnetseg'] = {
  "ratio" : [0.5, 0.25],
  "radius" : [100, 200],
  # 2 = dim (2) + dim of feature (0)
  # 130 = 128 + dim (2) + dim of feature (0)
  # 258 = 256 + dim (2) + dim of feature (0)
  "sa_channels" : [[2, 64, 64, 128], [130, 128, 128, 256], [258, 256, 512, 1024]],
  "k" : [1,3,3],
  # 1280 = 1024 + 256 
  # 384 = 256 + 128
  # 130 = 128 + dim of feature (0) 
  "fp_channels" : [[1280, 256, 256], [384, 256, 128], [128, 128, 128, 128]],
  # num classes = 2
  "output_channels": [128, 128, 128, 2],
  "dropout" : 0.2,
  "norm" : None,
}

# what trying to predict
config["label_level"] = "node" # graph

# train/val transforms
# options: ['normalisescale', 'jitter', 'flip', 'randscale', 'rotate', 'shear'] # null
config["transforms"] = ['normalisescale']

# wandb parameters
config["wandb_project"] = "nieves_expt1"
config["wandb_dataset"] = "nieves"

yaml_save_loc = os.path.join(input_folder, 'config/expt1/train.yaml')
with open(yaml_save_loc, "w") as outfile:
        yaml.dump(config, outfile)

# 1. Choice of augmentations

# 2. Normalise features


In [ ]:
# Run the train script 

!python ../src/locpix_points/scripts/train.py -i ../../../output/nieves/expt1 -c ../../../data/nieves/config/expt1/train.yaml

/home/oliver/mambaforge/envs/locpix_points/lib/python3.11/site-packages/torch_geometric/data/dataset.py:217: UserWarning: The `pre_filter` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-fitering technique, make sure to delete '{self.processed_dir}' first
  warnings.warn(
/home/oliver/smlm_cloud/locpix-points/src/locpix_points/models/point_net.py:197: UserWarning: There are redundant linear layers here...
  warnings.warn("There are redundant linear layers here...")


---- Params -----


Input features:  0
Num classes:  2
Batch size:  1
Epochs:  1
Number train graphs 665
Number val graphs 95
wandb: Currently logged in as: oliver-umney (teststest). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in /home/oliver/smlm_cloud/locpix-points/exp

### Evaluate

In [ ]:
# Export config variables

# Run the evaluate script

python src/locpix_points/scripts/evaluate.py -i ../../output/nieves -c src/locpix_points/templates/evaluate.yaml

### Interpret the results

In [13]:
# Pytorch geometric explainability tools